In [3]:
from os import listdir, walk
from os.path import isfile, join
import pandas as pd
import datetime

In [4]:
path = 'data/forex/oanda/minute/'
currencies = ['eurusd/'] #, 'audchf/']

In [5]:
files = []
for currency in currencies:
    files.append([f for f in listdir(path + currency) if 2016 <= int(f.split('_')[0][:4]) <= 2022])

In [6]:
files

[['20160103_quote.zip',
  '20160104_quote.zip',
  '20160105_quote.zip',
  '20160106_quote.zip',
  '20160107_quote.zip',
  '20160108_quote.zip',
  '20160110_quote.zip',
  '20160111_quote.zip',
  '20160112_quote.zip',
  '20160113_quote.zip',
  '20160114_quote.zip',
  '20160115_quote.zip',
  '20160117_quote.zip',
  '20160118_quote.zip',
  '20160119_quote.zip',
  '20160120_quote.zip',
  '20160121_quote.zip',
  '20160122_quote.zip',
  '20160124_quote.zip',
  '20160125_quote.zip',
  '20160126_quote.zip',
  '20160127_quote.zip',
  '20160128_quote.zip',
  '20160129_quote.zip',
  '20160131_quote.zip',
  '20160201_quote.zip',
  '20160202_quote.zip',
  '20160203_quote.zip',
  '20160204_quote.zip',
  '20160205_quote.zip',
  '20160207_quote.zip',
  '20160208_quote.zip',
  '20160209_quote.zip',
  '20160210_quote.zip',
  '20160211_quote.zip',
  '20160212_quote.zip',
  '20160214_quote.zip',
  '20160215_quote.zip',
  '20160216_quote.zip',
  '20160217_quote.zip',
  '20160218_quote.zip',
  '20160219_quot

In [7]:
import datetime
def convert_to_time(t):
    t /= 1000
    seconds = int(t%60)
    t /= 60
    minutes = int(t%60)
    t /= 60
    hours = int(t%24)
    return datetime.time(hours, minutes, seconds)

In [8]:
import talib

INDICATORS = {
    'BBANDS': {'method': 'BBANDS', 'period': 20}, 'SMA_SLOW': {'method': 'SMA', 'period': 200},
    'SMA_FAST': {'method': 'SMA', 'period': 50}, 'EMA_SLOW': {'method': 'EMA', 'period': 20},
    'EMA_FAST': {'method': 'EMA', 'period': 9}, 'ATR': {'method': 'ATR', 'period': 14},
    'RSI': {'method': 'RSI', 'period': 14}
}

def get_indicators(df, frame='1min'):
    df = df.copy()
    opens = df[f'open_{frame}'].values
    highs = df[f'high_{frame}'].values
    lows = df[f'low_{frame}'].values
    closes = df[f'close_{frame}'].values
    # Add indicator columns to DataFrame
    for indicator_name, indicator_values in INDICATORS.items():
        if indicator_name == 'BBANDS':
            ub, _, lb = getattr(talib, indicator_values['method'])(closes, indicator_values['period'])
            df[f'{indicator_name}_UB_{frame}'], df[f'{indicator_name}_LB_{frame}'] = ub, lb
        elif indicator_name == 'ATR':
            df[f"{indicator_name}_{frame}"] = \
                getattr(talib, indicator_values['method'])(highs,
                                                           lows,
                                                           closes,
                                                           indicator_values['period'])
        else:
            df[f"{indicator_name}_{frame}"] = getattr(talib, indicator_values['method'])(closes,
                                                                             indicator_values['period'])
    return df

In [137]:
import talib
PATTERNS = ["CDLSHOOTINGSTAR", "CDLDOJI", "CDLHAMMER", "CDLMORNINGSTAR", "CDLENGULFING", "CDLHARAMI", "CDLHANGINGMAN",
           "CDLDARKCLOUDCOVER", "CDLINVERTEDHAMMER", "CDLPIERCING", "CDLMORNINGDOJISTAR", "CDLDOJISTAR", "CDLEVENINGSTAR",
           "CDLEVENINGDOJISTAR", "CDLSPINNINGTOP", "CDL2CROWS", "CDL3BLACKCROWS", "CDL3WHITESOLDIERS", "CDLHARAMICROSS"]

def get_cdls(df, frame='1min'):
    df = df.copy()
    opens = df[f'open_{frame}'].values
    highs = df[f'high_{frame}'].values
    lows = df[f'low_{frame}'].values
    closes = df[f'close_{frame}'].values
    # Add indicator columns to DataFrame
    for pattern in PATTERNS:
        df[f"{pattern}_{frame}"] = getattr(talib, pattern)(opens, highs, lows, closes)/100
    return df

In [143]:
def cumsample(x):
    if x.shape[0] == 0:
        return pd.DataFrame()
    else:
        return pd.DataFrame({
            'open': [x['open_1min'].iloc[0]]*len(x), 
            'close': x['close_1min'], 
            'close_bid': x['close_bid_1min'], 
            'high': x['close_1min'].cummax(), 
            'low': x['close_1min'].cummin(),
            'day_high': x['day_high'].cummax(),
            'day_low': x['day_low'].cummin(),
            'previous_close': x['previous_close'].iloc[-1],
            'previous_high': x['previous_high'].iloc[-1],
            'previous_low': x['previous_low'].iloc[-1],
        })

In [19]:
train = pd.DataFrame()
test = pd.DataFrame()
sizes = []
temp_prev = None
for j, currency in enumerate(currencies):
    print(currency)
    for i, f in enumerate(files[j]):
        if (i+1) % 100 == 0:
            print(i)
        date = f.split('_')[0]
        temp = pd.read_csv(path+currency+f, names=['time', 'open_bid', 'high_bid', 'low_bid', 'close_bid', 'volume', 'open_ask', 'high_ask', 'low_ask', 'close_ask', 'volume2'])
        sizes.append(temp.shape[0])
        del temp['volume']
        del temp['volume2']
        temp['date'] = date
        temp['time'] = temp['time'].apply(convert_to_time)
    #     temp = temp[temp['time'].apply(lambda x: datetime.time(12) < x < datetime.time(20))]
        temp['datetime'] = temp['time'].apply(lambda x: datetime.datetime(int(date[:4]), int(date[4:6]), int(date[6:]), x.hour, x.minute, x.second, tzinfo=datetime.timezone.utc))

        symbol = currency[:3].upper()
        temp['spread'] = temp['close_ask'] - temp['close_bid']
        for c in temp.columns:
            if c.endswith('ask'):
                temp[c[:-4]+'_1min'] = temp[c]
                del temp[c]
            elif c.endswith('bid'):
                if c.startswith('close'):
                    temp[c+'_1min'] = temp[c]
                    del temp[c]
                else:
                    del temp[c]
                    
        temp['day_high'] = temp['close_1min'].cummax()
        temp['day_low'] = temp['close_1min'].cummin()
        if temp_prev is not None:
            temp['previous_close'] = temp_prev['close_1min'].iloc[-1]
            temp['previous_high'] = temp_prev['close_1min'].max()
            temp['previous_low'] = temp_prev['close_1min'].min()
        else:
            temp['previous_close'] = None
            temp['previous_high'] = None
            temp['previous_low'] = None
            
        temp_prev = temp.copy()
                    
        if f.startswith('2021'):
            test = pd.concat([test, temp])
        else:
            train = pd.concat([train, temp])

eurusd/
99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599


In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [135]:
train = train.set_index('datetime').resample('1T').fillna('ffill')
train['datetime'] = train.index
train = train[[row['datetime'].date().isoformat().replace('-', '') == row['date'] for _, row in train.iterrows()]]

test = test.set_index('datetime').resample('1T').fillna('ffill')
test['datetime'] = test.index
test = test[[row['datetime'].date().isoformat().replace('-', '') == row['date'] for _, row in test.iterrows()]]

In [ ]:
train = get_indicators(train, frame='1min')
test = get_indicators(test, frame='1min')

In [138]:
train = get_cdls(train, frame='1min')
test = get_cdls(test, frame='1min')

In [144]:
timeframes = ['5min', '15min', '30min', '60min']

dfs = pd.DataFrame()
for frame in timeframes:
    print(frame)
    df = train.resample(frame).apply(cumsample)
    if type(df.index) == pd.core.indexes.multi.MultiIndex:
        df.index = df.index.droplevel()
    df.columns = [c+'_'+frame for c in df.columns]
    df = get_indicators(df, frame=frame)
    df = get_cdls(df, frame=frame)
    dfs = pd.concat([dfs, df], axis=1)
train2 = pd.concat([train, dfs], axis=1)

dfs = pd.DataFrame()
for frame in timeframes:
    df = test.resample(frame).apply(cumsample)
    if type(df.index) == pd.core.indexes.multi.MultiIndex:
        df.index = df.index.droplevel()
    df.columns = [c+'_'+frame for c in df.columns]
    df = get_indicators(df, frame=frame)
    df = get_cdls(df, frame=frame)
    dfs = pd.concat([dfs, df], axis=1)
test2 = pd.concat([test, dfs], axis=1)

5min
15min
30min
60min


In [145]:
train2[[c for c in train2.columns if c.startswith('CDL')]].max()

CDLSHOOTINGSTAR_1min        0.0
CDLDOJI_1min                1.0
CDLHAMMER_1min              1.0
CDLMORNINGSTAR_1min         1.0
CDLENGULFING_1min           1.0
CDLHARAMI_1min              1.0
CDLHANGINGMAN_1min          0.0
CDLDARKCLOUDCOVER_1min      0.0
CDLINVERTEDHAMMER_1min      1.0
CDLPIERCING_1min            1.0
CDLMORNINGDOJISTAR_1min     1.0
CDLDOJISTAR_1min            1.0
CDLEVENINGSTAR_1min         0.0
CDLEVENINGDOJISTAR_1min     0.0
CDLSPINNINGTOP_1min         1.0
CDL2CROWS_1min              0.0
CDL3BLACKCROWS_1min         0.0
CDL3WHITESOLDIERS_1min      1.0
CDLHARAMICROSS_1min         1.0
CDLSHOOTINGSTAR_5min        0.0
CDLDOJI_5min                1.0
CDLHAMMER_5min              1.0
CDLMORNINGSTAR_5min         1.0
CDLENGULFING_5min           1.0
CDLHARAMI_5min              1.0
CDLHANGINGMAN_5min          0.0
CDLDARKCLOUDCOVER_5min      0.0
CDLINVERTEDHAMMER_5min      0.0
CDLPIERCING_5min            1.0
CDLMORNINGDOJISTAR_5min     1.0
CDLDOJISTAR_5min            1.0
CDLEVENI

In [148]:
train2.dropna().shape[0]/train2.shape[0]

0.9998952239495937

In [164]:
train = train2.copy()
test = test2.copy()

In [165]:
len(train), len(test)

(1899289, 117180)

In [166]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [167]:
len(train), len(test)

(1899090, 116981)

In [168]:
train.to_csv('train_timeframes_indicators2.csv', index=True)
test.to_csv('test_timeframes_indicators2.csv', index=True)